# Предсказание бойца - победителя UFC
## Расчет прибыльности
### Межгалактический хакатон 2022
Команда "Meldonium" <br/>
Состав: Вячеслав Барков, Павел Мамаев, Сергей Глуховский, Алексей Недоливко, Андрей Рем, Иван Ершов

In [3]:
import numpy as np
from flask import Flask, request, render_template
import pickle

In [4]:
!curl -L -sS https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb -o cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb &> /dev/null

In [5]:
path = 'predictions.bin'
with open(path, 'rb') as f:
    data  = pickle.load(f)

In [6]:
!bash /content/gdrive/MyDrive/.config/.cloudflared/install.sh

bash: /content/gdrive/MyDrive/.config/.cloudflared/install.sh: No such file or directory


In [6]:
!cp -r /content/gdrive/MyDrive/hackathon/templates/* /content/

In [74]:
def model(amount, use_model_strategy=True):
    if use_model_strategy:
        bank = 0
        for ind in data.index:
            winner, y_pred, odd1, odd2 = data.loc[ind, ['winner', 'prediction', 'f1_odds', 'f2_odds']]
            if winner == y_pred:
                if winner:
                    win_odd = odd1
                else:
                    win_odd = odd2
            else:
                win_odd = -1
            bank += amount * win_odd
        return bank
    else:
        bank = 0
        for ind in data.index:
            winner, y_pred, odd1, odd2 = data.loc[ind, ['winner', 'prediction', 'f1_odds', 'f2_odds']]
            favourite = min(odd1, odd2)
            if ((favourite == odd1) and winner) or ((favourite == odd2) and ~winner):
                win_odd = favourite
            else: 
                win_odd = 0
            bank += amount*win_odd - amount
        return bank

In [8]:
app = Flask(__name__, template_folder='/content')

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    amount, strategy = request.form.values()
    strategy = strategy == 'YES'
    print(amount, strategy)
    output = model(int(amount), strategy)

    if output > 0:
        output = f'Вы бы могли заработать ${output}!'
    else:
        output = f'К сожалению вы бы ничего не заработали и ушли в минус :(  ${output}!'

    return render_template('index.html', prediction_text=output)


app.run(port=80)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:80/ (Press CTRL+C to quit)
127.0.0.1 - - [20/May/2022 20:26:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2022 20:26:06] "GET /favicon.ico HTTP/1.1" 404 -


100 True


127.0.0.1 - - [20/May/2022 20:26:15] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2022 20:28:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2022 20:28:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2022 20:28:30] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2022 20:28:30] "GET /favicon.ico HTTP/1.1" 404 -
[2022-05-20 20:28:42,426] ERROR in app: Exception on /predict [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/a

100 True


...

127.0.0.1 - - [20/May/2022 20:29:36] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2022 20:30:02] "GET / HTTP/1.1" 200 -


100 True


127.0.0.1 - - [20/May/2022 20:30:15] "POST /predict HTTP/1.1" 200 -


100 False


127.0.0.1 - - [20/May/2022 20:30:25] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2022 20:31:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2022 20:31:06] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2022 20:31:07] "GET /favicon.ico HTTP/1.1" 404 -
